In [7]:
import pandas as pd
from datetime import datetime

# 1) Parameters
country_code = 'US'                        # e.g. 'US', 'GB', 'CN', etc.
start_year   = 1960
end_year     = datetime.now().year

# 2) Construct the CSV‐download URL
csv_url = (
    f"https://api.worldbank.org/v2/country/{country_code}"
    f"/indicator/NY.GDP.MKTP.KD.ZG"
    f"?date={start_year}:{end_year}"
    f"&format=csv&per_page=1000"
)

# 3) Read directly into pandas, skipping the metadata rows
#    The first 4 rows are header information, then row 5 is the column names.
df_wide = pd.read_csv(csv_url, skiprows=4)

# 4) (Optional) Save the raw CSV for later
df_wide.to_csv(f'gdp_growth_{country_code}.csv', index=False)
print(f"Saved raw data to gdp_growth_{country_code}.csv")

# 5) Melt the “wide” table into a time‐series
df = (
    df_wide
    .melt(
        id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'],
        var_name='Year',
        value_name='GDP Growth (%)'
    )
)
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df = df[['Year','GDP Growth (%)']].sort_values('Year').reset_index(drop=True)

# 6) Inspect
print(df.head())


# 6) Set up the Matplotlib figure
fig, ax = plt.subplots(figsize=(8,4))
ax.set_title(f"{country_code} GDP Growth (annual %), {start_year}–{end_year}")
ax.set_xlabel("Year")
ax.set_ylabel("GDP Growth (%)")
ax.set_xlim(years.min(), years.max())
ax.set_ylim(values.min() - 1, values.max() + 1)

line, = ax.plot([], [], lw=2)

# 7) Animation callbacks
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    line.set_data(years[:i+1], values[:i+1])
    return (line,)

anim = FuncAnimation(
    fig, animate, init_func=init,
    frames=len(years),
    interval=200,   # milliseconds between frames → ~5 fps
    blit=True,
    repeat=True
)

# 8) Display inline as HTML5 animation
HTML(anim.to_jshtml())


EmptyDataError: No columns to parse from file